In [1215]:
def is_variable(pat):
    return pat.startswith('?') and not(pat == '?') and all(s.isalpha() for s in pat[1:]) #'?'不算作占位符

In [1216]:
is_variable('?')

False

In [1217]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [1218]:
pat_match('I like ?x'.split(), 'I like reading'.split())

True

In [1219]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [1220]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [1221]:
pat_match(pattern, saying)

('?X', 'holiday')

In [1222]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

In [1223]:
def pat_match(pattern, saying):
    result = [] 
    if not(len(pattern) == len(saying)):
        return []
    else:
        for i in range(len(pattern)):
            if (pattern[i] != saying[i]):
                if not is_variable(pattern[i]):
                    return []
                else:
                    result += [(pattern[i], saying[i])]
        
        return result
    """
    if not pattern or not saying: return [] #模式不匹配
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return [] #模式不匹配
        else:
            return pat_match(pattern[1:], saying[1:])
    """

In [1224]:
pat_match("?X greater than ?Y".split(), "3 greater than 2 ".split())

[('?X', '3'), ('?Y', '2')]

In [1225]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [1226]:
pat_to_dict([('?X', '3'), ('?Y', '2')])

{'?X': '3', '?Y': '2'}

In [1227]:
def substitute(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules) #若某个单词不在parsed_rules中，返回该单词

1.通过问句与模式匹配，获取变量替换规则

In [1228]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [1229]:
got_patterns

[('?X', 'iPhone')]

2.将答句中变量替换成规则中该变量对应的单词

In [1230]:
substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [1231]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [1232]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"],
    "I want ?*P china ?*X with my ?X": ["Why do you want ?*P china ?*X with your ?X ?", "How do you ?*P china ?*X with your ?X ?"]
}

In [1233]:
import random

In [1234]:
def get_response(saying, rules):
    for k in rules:
        
        got_patterns = pat_match(k.split(), saying.split()) #通过saying获得模式匹配关系
        answer = random.choice(rules.get(k))
        
        if not got_patterns: continue
        else:
            return ' '.join(substitute(answer.split(),pat_to_dict(got_patterns)))
        #print ('\n')
        #print (k)
        #print(answer)
        #print(pat_match(k.split(), saying.split()))
        #print( ' '.join(substitute(answer.split(),pat_to_dict(got_patterns))))

In [1235]:
get_response("I need  iPhone",defined_patterns)

'Image you will get iPhone soon'

In [1236]:
get_response("My friend told me something",defined_patterns)

'How do you think about your friend ?'

In [1237]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and len(pattern) > 2 and all(a.isalpha() for a in pattern[2:]) #'?*'不算作占位符

In [1238]:
is_pattern_segment('?*P')

True

In [1239]:
from collections import defaultdict

In [1240]:
def segment_match(pattern,saying):
    if len(pattern) > 1:
        index = saying.index(pattern[1])
        return ' '.join(s for s in saying[:index])
    else:
        return ' '.join(s for s in saying)

In [1241]:
segment_match("?*P".split(), "travel to china next year with my friend".split())

'travel to china next year with my friend'

In [1242]:
segment_match("?*P china ?*X with my ?X".split(), "travel to china next year with my friend".split())

'travel to'

In [1243]:
#fail = [True, None]
"""
input:parttern.split(), saying.split()
I want ?*P china ?*X with my ?X
I want travel to china next year with my friend

output
[('?*P', ' travel to'), ('?*X', 'next year'), ('?X', 'friend')]
"""
def pat_match_with_seg(pattern, saying):
    result = []
    j = 0
    for i in range(len(pattern)):
        if j > len(saying) - 1: #pattern还有未匹配字符串，但是saying已无可匹配字符串
            return []
        elif is_variable(pattern[i]):
            result += [(pattern[i], saying[j])]
            j += 1
        elif is_pattern_segment(pattern[i]):
            seg = segment_match(pattern[i:], saying[j:])
            j = j + len(seg.split())
            result += [(pattern[i], seg)]
        elif pattern[i] == saying[j]:
            j += 1
            continue
        else:
            return []
    return result
    """
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    """


In [1253]:
pat_match_with_seg("I want ?*P china ?*X with my ?X".split(), "I want go to china next year with my friend".split())

[('?*P', 'go to'), ('?*X', 'next year'), ('?X', 'friend')]

In [1245]:
def get_response(saying, rules):
    for k in rules:
        
        got_patterns = pat_match_with_seg(k.split(), saying.split()) #通过saying获得模式匹配关系
        #print(got_patterns)
        answer = random.choice(rules.get(k))
        
        if not got_patterns: continue
        else:
            return ' '.join(substitute(answer.split(),pat_to_dict(got_patterns)))

In [1252]:
get_response("I want travel to china next year with my friend",defined_patterns)

[]
[]
[('?*P', 'travel to'), ('?*X', 'next year'), ('?X', 'friend')]


'Why do you want travel to china next year with your friend'